In [127]:
import pandas as pd
import sqlalchemy
import json
import datetime as dt
import html5lib
import lxml
from bs4 import BeautifulSoup as bs4
from sqlalchemy import create_engine


In [137]:
rds_connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [2]:
gun_permits_csv = "federal-firearms-licenses.csv"
gun_violence_csv = "gun-violence-data_01-2013_03-2018.csv"
gun_p = "0118-ffl-list.csv"
pop_url = "https://simple.wikipedia.org/wiki/List_of_U.S._states"
firearms = "firearms.xlsx"

In [3]:
gun_permit_df = pd.read_csv(gun_permits_csv)
gun_violence_df = pd.read_csv(gun_violence_csv)
population = pd.read_html(pop_url)
firearm = pd.read_excel(firearms, header=2)

C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2,3,4,5,6,7,10,12,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#view columns
gun_permit_df.columns

Index(['license_state', 'license_status', 'license_number', 'license_type',
       'additional_license', 'additional_license_type', 'license_start_date',
       'license_expire_date', 'licensee_name_business', 'licensee_principal',
       'establishment_address_full', 'establishment_address_street1',
       'establishment_address_street2', 'establishment_address_city',
       'establishment_address_state', 'establishment_address_zip',
       'licensee_address_full', 'licensee_address_street1',
       'licensee_address_street2', 'licensee_address_city',
       'licensee_address_state', 'licensee_address_zip', 'establishment_type',
       'email', 'phone', 'serialid'],
      dtype='object')

In [46]:
gun_permit_df.head(15)

,license_state,license_status,license_number,license_type,additional_license,additional_license_type,license_start_date,license_expire_date,licensee_name_business,licensee_principal,...,licensee_address_full,licensee_address_street1,licensee_address_street2,licensee_address_city,licensee_address_state,licensee_address_zip,establishment_type,email,phone,serialid
0,Alaska,NaN,1,Common Carrier - Single Destination,NaN,NaN,NaN,2017-01-01,Alaska Airlines - Multiple,"Alaska Airlines, Inc.",...,PO Box 68900 SEAZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206-392-5875,1
1,Alaska,NaN,3,Beverage Dispensary,NaN,NaN,NaN,2007-01-01,Acres Bar,Acres Lounge Inc,...,PO Box 1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(907) 835-2269,2
2,Alaska,NaN,5,Beverage Dispensary-Tourism AS 04.11.400(d),NaN,NaN,NaN,NaN,"AHTNA, THE",AHTNA DEVELOPMENT CORPORATION,...,DRAWER G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,Alaska,NaN,6,Beverage Dispensary-Tourism AS 04.11.400(d),NaN,NaN,NaN,2016-01-01,Airport Lounge & Snack Bar,Airport Lounge & Snack Bar LLC,...,PO Box 8256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(907) 225-6455,4
4,Alaska,NaN,7,Package Store,NaN,NaN,NaN,NaN,AIRPORT ROAD LIQUOR STORE,M & R INC,...,3206 AIRPORT RD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5,Alaska,NaN,8,Package Store,NaN,NaN,NaN,2018-01-01,Alaska Commercial Company,"The North West Company (International), Inc.",...,77 Main Street Attn. Legal Dept.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(907) 383-3111 204-934-1705; 907-273-4642,6
6,Alaska,NaN,10,Beverage Dispensary-Tourism AS 04.11.400(d),NaN,NaN,NaN,2017-01-01,Al's Alaskan Inn,Allen Henry Choy LLC,...,7830 Old Seward Hwy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(907) 440-9176,7
7,Alaska,NaN,11,Common Carrier - Single Destination,NaN,NaN,NaN,2017-01-01,Alaska Airlines - Adak,"Alaska Airlines, Inc.",...,PO Box 68900 SEAZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206-392-5875,8
8,Alaska,NaN,12,Common Carrier - Single Destination,NaN,NaN,NaN,2017-01-01,Alaska Airlines - Anchorage,"Alaska Airlines, Inc.",...,PO Box 68900 SEAZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206-392-5875,9
9,Alaska,NaN,13,Common Carrier - Single Destination,NaN,NaN,NaN,2017-01-01,Alaska Airlines - Kodiak,"Alaska Airlines, Inc.",...,PO Box 68900 SEAZL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206-392-5875,10


In [6]:
gun_permit_df.license_state.value_counts()

Louisiana                    1070734
Ohio                          360847
North Carolina                173246
Florida                       165953
Georgia                       119756
California                    114501
Texas                          62515
New York                       51511
Arizona                        35070
Minnesota                      30424
Illinois                       28158
Alabama                        27311
Pennsylvania                   25407
New Hampshire                  21781
Indiana                        20681
South Carolina                 20204
Iowa                           19487
Oregon                         19422
Virginia                       18583
Missouri                       17034
Wisconsin                      16975
Michigan                       16042
West Virginia                  14425
Colorado                       14316
Kentucky                       14067
Washington - ON                10381
Washington - OFF                9118
N

In [7]:
#retrieve wanted columns
gun_permit_df.head()
gun_permit_cleaning = gun_permit_df[['license_state', 'license_start_date', 'license_expire_date']].copy()

In [8]:
#drop NA from start & expire date column
gun_permit_cleaning.dropna(subset=['license_start_date', 'license_expire_date'], inplace=True)

In [9]:
#filter for dates after 2013-01-01
gun_permit = gun_permit_cleaning.loc[(gun_permit_cleaning['license_start_date'] >= '2013-01-01') 
                                            & (gun_permit_cleaning['license_start_date'] < '2018-01-01')]

In [10]:
#create new table from gun permit that contains 50 states and counts 
state_table_permit = gun_permit.groupby('license_state').count()


In [11]:
#rename and drop
state_table_permit = state_table_permit.rename(columns={'license_start_date':'Permits_Issued'})
state_table_permit.drop(columns=['license_expire_date'], inplace=True)

In [12]:
state_table_permit

,Permits_Issued
license_state,
Arizona,5385
California,36666
Connecticut,3372
Florida,93499
Georgia,85393
Idaho,4870
Illinois,28030
Indiana,5230
Iowa,5766


In [13]:
gun_violence_df.date.dropna(inplace = True)


In [14]:
gun_violence_df['date'] = pd.to_datetime(gun_violence_df['date'])

In [15]:
gun_violence_df.dtypes

incident_id                             int64
date                           datetime64[ns]
state                                  object
city_or_county                         object
address                                object
n_killed                                int64
n_injured                               int64
incident_url                           object
source_url                             object
incident_url_fields_missing              bool
congressional_district                float64
gun_stolen                             object
gun_type                               object
incident_characteristics               object
latitude                              float64
location_description                   object
longitude                             float64
n_guns_involved                       float64
notes                                  object
participant_age                        object
participant_age_group                  object
participant_gender                

In [17]:
gun_violence = gun_violence_df[['date', 'state', 'n_killed', 'n_injured' ]].copy()

In [18]:
#create table of states and count incidents
state_violence = gun_violence.groupby('state').count()
state_violence = state_violence.rename(columns={'date':'Incidents'})

#change values in columns to match number killed and number injured
state_violence['n_killed'] = gun_violence.groupby('state').sum()['n_killed']
state_violence['n_injured'] = gun_violence.groupby('state').sum()['n_injured']

In [74]:
state_violence.rename(columns={'n_killed':'#Killed',
                            'n_injured':'#Injured'}, 
                     inplace=True)
state_violence.rename_axis("State", axis='index', inplace=True)

In [75]:
state_violence.head()

,Incidents,#Killed,#Injured
State,,,
Alabama,5471,1880,2998
Alaska,1349,267,325
Arizona,2328,1094,1096
Arkansas,2842,773,1347
California,16306,5562,7644


In [31]:
#html imported as a dataframe in a list, seperate datafram
pop =  population[0]

#copy only columns wanted
pop_clean = pop[['Name &postal abbreviation[1]', 'Population[upper-alpha 2][3]']]

#rename columns
pop_clean = pop_clean.rename(columns={'Name &postal abbreviation[1]':'State', 
                                     'Population[upper-alpha 2][3]':'Population'})

In [34]:
#extract columns and create a new dataframe
#I did this to do away with the weird multi-indexing and subcolumns issues
pop_clean.State = pop_clean.State.astype(str)
state = []
for i, row in pop_clean.iterrows():
    state.append([row[0], row[2]])

Population = pd.DataFrame(state, columns=['State',  'Population'])


In [35]:
#clean state names
st = []
for list in Population.State.str.split("["):
    st.append(list[0])

Population.State = st

In [38]:
Population.head()

,State,Population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [ ]:
#drop unwanted columns
firearm.drop([51, 52, 53, 54], axis=0, inplace=True)
firearm.drop(['Unnamed: 8'], axis=1, inplace=True)

In [28]:
firearm.drop([7], axis=0, inplace=True)

In [40]:
Population.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [96]:
State_db = pd.merge(Population, firearm, how='inner', on='State')


In [97]:
State_db.set_index('State', inplace=True)

In [98]:
State_final_db = pd.merge(State_db, state_violence, how='left', on='State')

In [99]:
State_final_db.columns

Index(['Population', 'Any Other Weapon1', 'Destructive Device2', 'Machinegun3',
       'Silencer4', 'Barreled Rifle5', 'Barreled Shotgun6', 'Total',
       'Incidents', '#Killed', '#Injured'],
      dtype='object')

In [100]:
State_final_db.rename(columns={'Any Other Weapon1':'Other_Weapon', 
                               'Destructive Device2':'Destructive_Device', 
                               'Machinegun3':'Machinegun',
                               'Silencer4':'Silencer',
                               'Barreled Rifle5':'Barreled_Rifle',
                               'Barreled Shotgun6':'Barreled_Shotgun',
                               'Total':'Total_Weapons'},
                                inplace=True)

In [108]:
State_final_db.dtypes

Population              int64
Other_Weapon          float64
Destructive_Device    float64
Machinegun            float64
Silencer              float64
Barreled_Rifle        float64
Barreled_Shotgun      float64
Total_Weapons         float64
Incidents               int64
#Killed                 int64
#Injured                int64
dtype: object

In [124]:
#Convert all values to float
State_final_db = State_final_db.apply(pd.to_numeric, axis=1)

In [155]:
State_final_db.rename(columns={'#Killed':'people_killed', '#Injured':'people_injured'})
State_final_db.columns = State_final_db.columns.str.lower()

In [126]:

#write file to csv 
output_File = "ETL_csv.csv"
State_final_db.to_csv(output_File, index=False)
State_final_db
pd.

In [156]:
#write to postgresql database
State_final_db.to_sql(name = 'state_data', con=engine, if_exists='replace')

In [158]:
#query and read postgres database
pd.read_sql_query('Select * from state_data', con=engine)

,State,population,other_weapon,destructive_device,machinegun,silencer,barreled_rifle,barreled_shotgun,total_weapons,incidents,#killed,#injured
0,Alabama,4903185.0,1228.0,80726.0,29757.0,46902.0,7214.0,2438.0,168265.0,5471.0,1880.0,2998.0
1,Alaska,731545.0,332.0,5346.0,1674.0,9250.0,2503.0,1415.0,20520.0,1349.0,267.0,325.0
2,Arizona,7278717.0,1365.0,108692.0,17841.0,55099.0,19124.0,2696.0,204817.0,2328.0,1094.0,1096.0
3,Arkansas,3017804.0,651.0,70179.0,5657.0,26804.0,4279.0,1231.0,108801.0,2842.0,773.0,1347.0
4,California,39512223.0,4113.0,299766.0,30173.0,15131.0,13354.0,14129.0,376666.0,16306.0,5562.0,7644.0
5,Colorado,5758736.0,1045.0,50349.0,7377.0,41983.0,10008.0,1929.0,112691.0,3201.0,796.0,1133.0
6,Connecticut,3565278.0,973.0,13647.0,41920.0,13731.0,3567.0,1039.0,74877.0,3067.0,341.0,1258.0
7,Delaware,973764.0,35.0,3324.0,542.0,388.0,357.0,635.0,5281.0,1685.0,217.0,853.0
8,Florida,21477737.0,3881.0,216851.0,45893.0,118097.0,38216.0,9643.0,432581.0,15029.0,3909.0,7072.0
9,Georgia,10617423.0,2113.0,80158.0,33743.0,82003.0,16238.0,11738.0,225993.0,8925.0,2456.0,4056.0
